# Debugging Script 
This is a manual script of the functions for debugging purposes

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
#Libraries 
import os
import pyodbc
import glob
import copy
import PySimpleGUI as sg
import traceback

# Bring in Part 1 Code

In [2]:
!pip -q install -r ./config_files/requirements.txt
import pandas as pd
pd.options.mode.chained_assignment = None

In [3]:
# Bring in functions
%run ./scripts/Part_1.ipynb

OperationalError: ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]Named Pipes Provider: Could not open a connection to SQL Server [53].  (53) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (53)')

OperationalError: ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]Named Pipes Provider: Could not open a connection to SQL Server [53].  (53) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (53)')

 # Individual Files

In [4]:
# inputs 
ds_ID = 'DS42'

comparison_processed_data vs comparison_no_geozone_df:
- comparison_processed_data has columns appending the DSID and has the geozone at the end (jurisdiction level)
- features are all of teh columns


In [5]:
# mgra_ind
comparison_processed_data, comparison_no_geozone_df, features = download_DS_data(ds_ID, jur_level)

DS42's jurisdiction did not merge correctly.


In [6]:
# CPA Level Aggregation
comparison_processed_data_cpa = cpa_aggregation_ind(comparison_no_geozone_df, cpa_level)

In [7]:
# Jurisdiction aggregation
comparison_processed_data_jur = jur_aggregation_ind(comparison_no_geozone_df, jur_level)

In [8]:
# Region Aggregation
comparison_processed_data_reg = region_aggregation_ind(comparison_no_geozone_df)

# Comparison Files

In [9]:
# Inputs for comparison 
first_ID = "DS38"
second_ID = 'DS41'

In [10]:
# Download the necessary data
# First ID
comparison_first_ID_processed_data, comparison_first_ID_no_geozone_df, first_ID_features = download_DS_data(first_ID, jur_level)

# Second ID
comparison_second_ID_processed_data, comparison_second_ID_no_geozone_df, second_ID_features = download_DS_data(second_ID, jur_level)

DS38's jurisdiction did not merge correctly.
DS41's jurisdiction did not merge correctly.


## Both Files

In [11]:
# MGRA_Both Files
# This does the right thing and drops 'secondary' not sure how it knows to do that but it does 
mgra_both = concat_dfs_temp(comparison_first_ID_no_geozone_df, comparison_second_ID_no_geozone_df)

In [12]:
# CPA both
cpa_both = cpa_aggregation(comparison_first_ID_no_geozone_df, comparison_second_ID_no_geozone_df, cpa_level)

In [13]:
# Jur both 
jur_both = jur_aggregation(comparison_first_ID_no_geozone_df, comparison_second_ID_no_geozone_df, jur_level)

In [14]:
# Region Both
region_both = region_aggregation(comparison_first_ID_no_geozone_df, comparison_second_ID_no_geozone_df)

## Diff Files

In [15]:
'''The issue is that in mgra_both 'secondary' does not exist '''

'''The issue with this code originally is that the input originally was create_diff(features_first_ID, features_second_ID, first_second_ID_comparison) and the features_first_ID had non numeric columns such as secondary and elementary while mgra-both only had numeric (because of the groupby at the end) so the way the code was written it was trying to subtract columns that were alike in 'features_first_ID' and 'features_second_ID' but some didn't exist in mgra_both'''

'''Create diff goes and keeps things in the same order, it doesn't use set. Now the code is grabbing columns from the both file itself. '''

mgra_diff = create_diff(list([x[:-5] for x in mgra_both if x[-5:] == '_' + first_ID]), list([x[:-5] for x in mgra_both if x[-5:] == '_' + second_ID]), mgra_both)

C:\Users\cra\AppData\Local\Temp\ipykernel_20132\564207998.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  diff_df[f'{column}_diff'] = first_second_ID_comparison[f'{column}_{first_ID}'] - first_second_ID_comparison[f'{column}_{second_ID}']


In [16]:
cpa_diff = create_diff_temp(list([x[:-5] for x in cpa_both if x[-5:] == '_' + first_ID]),list([x[:-5] for x in cpa_both if x[-5:] == '_' + second_ID]), cpa_both) 

C:\Users\cra\AppData\Local\Temp\ipykernel_20132\3044449395.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  diff_df[f'{column}_diff'] = first_second_ID_comparison[f'{column}_{first_ID}'] - first_second_ID_comparison[f'{column}_{second_ID}']


In [17]:
jur_diff = create_diff_temp(list([x[:-5] for x in jur_both if x[-5:] == '_' + first_ID]), list([x[:-5] for x in jur_both if x[-5:] == '_' + second_ID]), jur_both)

C:\Users\cra\AppData\Local\Temp\ipykernel_20132\3044449395.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  diff_df[f'{column}_diff'] = first_second_ID_comparison[f'{column}_{first_ID}'] - first_second_ID_comparison[f'{column}_{second_ID}']


In [18]:
region_diff = create_diff_temp(list([x[:-5] for x in region_both if x[-5:] == '_' + first_ID]), list([x[:-5] for x in region_both if x[-5:] == '_' + first_ID]), region_both)

C:\Users\cra\AppData\Local\Temp\ipykernel_20132\3044449395.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  diff_df[f'{column}_diff'] = first_second_ID_comparison[f'{column}_{first_ID}'] - first_second_ID_comparison[f'{column}_{second_ID}']
